In [19]:
include("forward_pass.jl")
include("backward_pass.jl");
include("utils.jl");
include("convolution.jl");
include("graph_building.jl");
include("load_data.jl");
include("scalar_operators.jl");
include("broadcasted_operators.jl")

backward (generic function with 15 methods)

In [20]:
x = Variable([5.0,4.0], name="x")
y = Variable([5.0,4.0], name="y")
z = x * y
two = Constant(2.0)
squared = z.^two
sine = sin(squared)

order = topological_sort(sine)

6-element Vector{Any}:
 var x
 ┣━ ^ 2-element Vector{Float64}
 ┗━ ∇ Nothing
 var y
 ┣━ ^ 2-element Vector{Float64}
 ┗━ ∇ Nothing
 op.?(typeof(mul!))
 const 2.0
 op.?(typeof(^))
 op ?(typeof(sin))

In [21]:
using MLDatasets, Base, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x4dim = reshape(data.features, 28, 28, 1, :)
    yhot  = Flux.onehotbatch(data.targets, 0:9) 
    Flux.DataLoader((x4dim, yhot); batchsize, shuffle=true)
end

loader (generic function with 1 method)

In [22]:
x1, y1 = first(loader(train_data))

(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;;], Bool[0; 0; … ; 0; 0;;])

In [23]:
function dense(w, b, x, activation) return activation(w * x .+ b) end
function dense(w, x, activation) return activation(w * x) end
function dense(w, x) return w * x end

function mean_squared_loss(y, ŷ)
    return Constant(0.5) .* (y .- ŷ) .^ Constant(2)
end
#=
net = Chain(
    Conv((3, 3), 1 => 6,  relu, bias=false),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(13*13*6 => 84, relu, bias=false), 
    Dense(84 => 10, identity, bias=false)
)
=#

mean_squared_loss (generic function with 1 method)

In [30]:
Wh  = Variable(randn(10,2), name="wh")
Wo  = Variable(randn(1,10), name="wo")
x = Variable([1.98, 4.434], name="x")
y = Variable([0.064], name="y")

var y
 ┣━ ^ 1-element Vector{Float64}
 ┗━ ∇ Nothing

In [31]:
function net(x, wh, wo, y)
    x̂ = dense(wh, x, relu)
    x̂.name = "x̂"
    ŷ = dense(wo, x̂)
    ŷ.name = "ŷ"
    E = mean_squared_loss(y, ŷ)
    E.name = "loss"

    return topological_sort(E)
end
graph = net(x, Wh, Wo, y)
forward!(graph)
backward!(graph)


In [32]:

for (i,n) in enumerate(graph)
    print(i, ". "); println(n)
end

1. const 0.5
2. var y
 ┣━ ^ 1-element Vector{Float64}
 ┗━ ∇ 1×1 Matrix{Float64}
3. var wo
 ┣━ ^ 1×10 Matrix{Float64}
 ┗━ ∇ 1×10 Matrix{Float64}
4. var wh
 ┣━ ^ 10×2 Matrix{Float64}
 ┗━ ∇ 10×2 Matrix{Float64}
5. var x
 ┣━ ^ 2-element Vector{Float64}
 ┗━ ∇ 2×1 Matrix{Float64}
6. op.?(typeof(mul!))
7. op.x̂(typeof(relu))
8. op.ŷ(typeof(mul!))
9. op.?(typeof(-))
10. const 2
11. op.?(typeof(^))
12. op.loss(typeof(*))
